In [ ]:
import os
if "COLAB_GPU" in os.environ:
    print("[INFO] Running in Google Colab, installing requirements.")
    !pip install -U torch
    !pip install PyMuPDF
    !pip install tqdm
    !pip install sentence-transformers
    !pip install accelerate
    !pip install bitsandbytes
    !pip install flash-attn --no-build-isolation

[INFO] Running in Google Colab, installing requirements.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9

In [ ]:
import os
import requests
pdf_path = "/content/sodapdf-converted.pdf"

In [ ]:
import fitz
from tqdm.auto import tqdm

def text_formatter(text: str) -> str:
    cleaned_text = text.replace("\n", " ").strip()
    return cleaned_text

def open_and_read_pdf(pdf_path: str) -> list[dict]:
    doc = fitz.open(pdf_path)
    pages_and_texts = []
    for page_number, page in tqdm(enumerate(doc)):
        text = page.get_text()
        text = text_formatter(text)
        pages_and_texts.append({"page_number": page_number - 0,
                                "page_char_count": len(text),
                                "page_word_count": len(text.split(" ")),
                                "page_sentence_count_raw": len(text.split(". ")),
                                "page_token_count": len(text) / 4,
                                "text": text})
    return pages_and_texts
pages_and_texts = open_and_read_pdf(pdf_path=pdf_path)
pages_and_texts[:2]

0it [00:00, ?it/s]

[{'page_number': 0,
  'page_char_count': 5237,
  'page_word_count': 922,
  'page_sentence_count_raw': 22,
  'page_token_count': 1309.25,
  'text': 'Managed by Mikel Arteta in his first full season, Arsenal finished eighth in their Premier League season,  and following eliminations from the FA Cup, EFL Cup and UEFA Europa League, they failed to qualify for  any European competition for the first time since the 1994–95 season.       Arsenal’s 2019–20 campaign had seen defensive frailties and a lack of creativity lead to a disappointing  eighth-place finish in the league, the club’s lowest for a quarter of a century, but a successful FA Cup run  and increasing promise under new coach and former cup-winning captain Mikel Arteta gave them  increased optimism for the season ahead. The season had been defined by a three-month lull between  March and June, caused by the COVID-19 pandemic; indeed, it was head coach Arteta’s positive test that led to the season’s pause, and when it returned, it 

In [ ]:
import random
random.sample(pages_and_texts, k=3)

[{'page_number': 105,
  'page_char_count': 4987,
  'page_word_count': 957,
  'page_sentence_count_raw': 7,
  'page_token_count': 1246.75,
  'text': "history after Chelsea midfielder Enzo Fernández (£105.6m).[94] English midfielder Alex  Oxlade-Chamberlain (31 August 2017) and American striker Folarin Balogun (30 August 2023) drew  Arsenal's record fee received for a player transfer when they joined Liverpool and Monaco respectively for £35m each.[92][93]       In 1928, with Bolton Wanderers in financial trouble, Herbert Chapman's Arsenal made David Jack the  first five-digit signing in world football, almost double the previous record; the final fee paid was £10,647  10 shillings.[95] According to Bob Wall, Chapman negotiated the transfer with Bolton's representatives in  a hotel bar, his tactic being to drink gin and tonics without any gin in them, while asking the waiter to  double the alcohol served to the other side. Chapman remained sober while the Bolton representatives  got very

In [ ]:
from spacy.lang.en import English
nlp = English()
nlp.add_pipe("sentencizer")

In [ ]:
for item in tqdm(pages_and_texts):
    item["sentences"] = list(nlp(item["text"]).sents)
    item["sentences"] = [str(sentence) for sentence in item["sentences"]]
    item["page_sentence_count_spacy"] = len(item["sentences"])

In [ ]:
num_sentence_chunk_size = 23
def split_list(input_list: list,
               slice_size: int) -> list[list[str]]:
    return [input_list[i:i + slice_size] for i in range(0, len(input_list), slice_size)]
for item in tqdm(pages_and_texts):
    item["sentence_chunks"] = split_list(input_list=item["sentences"],
                                         slice_size=num_sentence_chunk_size)
    item["num_chunks"] = len(item["sentence_chunks"])

  0%|          | 0/127 [00:00<?, ?it/s]

In [ ]:
random.sample(pages_and_texts, k=1)

[{'page_number': 93,
  'page_char_count': 5304,
  'page_word_count': 950,
  'page_sentence_count_raw': 13,
  'page_token_count': 1326.0,
  'text': 'celebrated as the engine of the team.[36] Three years after their first Cup final, in 1929–30, Arsenal  reached Wembley again, this time up against Chapman\'s old club Huddersfield Town. The match was  notable for being "buzzed" by the enormous German airship Graf Zeppelin. Arsenal were not distracted  from their task; they won 2–0 with goals from James and Lambert to bring home the club\'s first major  trophy.[37] Their FA Cup success was the first in a decade in which Arsenal were the dominant club in  England.[37] They won the First Division for the first time in 1930–31;[37] Arsenal performed strongly in a  free-scoring title race with Aston Villa, recording several heavy wins (including 7–1 against Blackpool, 7–2 against Leicester City, and 9–1 against Grimsby Town.[38] Arsenal finished the season as champions  having scored 127 league

In [ ]:
import re
pages_and_chunks = []
for item in tqdm(pages_and_texts):
    for sentence_chunk in item["sentence_chunks"]:
        chunk_dict = {}
        chunk_dict["page_number"] = item["page_number"]
        joined_sentence_chunk = "".join(sentence_chunk).replace("  ", " ").strip()
        joined_sentence_chunk = re.sub(r'\.([A-Z])', r'. \1', joined_sentence_chunk)
        chunk_dict["sentence_chunk"] = joined_sentence_chunk
        chunk_dict["chunk_char_count"] = len(joined_sentence_chunk)
        chunk_dict["chunk_word_count"] = len([word for word in joined_sentence_chunk.split(" ")])
        chunk_dict["chunk_token_count"] = len(joined_sentence_chunk) / 4
        pages_and_chunks.append(chunk_dict)
len(pages_and_chunks)

  0%|          | 0/127 [00:00<?, ?it/s]

245

In [ ]:
random.sample(pages_and_chunks, k=1)

[{'page_number': 93,
  'sentence_chunk': 'celebrated as the engine of the team.[36] Three years after their first Cup final, in 1929–30, Arsenal reached Wembley again, this time up against Chapman\'s old club Huddersfield Town. The match was notable for being "buzzed" by the enormous German airship Graf Zeppelin. Arsenal were not distracted from their task; they won 2–0 with goals from James and Lambert to bring home the club\'s first major trophy.[37] Their FA Cup success was the first in a decade in which Arsenal were the dominant club in England.[37] They won the First Division for the first time in 1930–31;[37] Arsenal performed strongly in a free-scoring title race with Aston Villa, recording several heavy wins (including 7–1 against Blackpool, 7–2 against Leicester City, and 9–1 against Grimsby Town.[38] Arsenal finished the season as champions having scored 127 league goals (another club record),[39] though runners-up Villa managed to score 128,[40] a record for the most goals i

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame(pages_and_chunks)
df.describe().round(2)

,page_number,chunk_char_count,chunk_word_count,chunk_token_count
count,245.00,245.00,245.00,245.00
mean,66.26,2346.24,410.97,586.56
std,36.04,1588.63,267.27,397.16
min,0.00,29.00,4.00,7.25
25%,37.00,996.00,177.00,249.00
50%,69.00,2005.00,355.00,501.25
75%,97.00,3494.00,621.00,873.50
max,126.00,5472.00,928.00,1368.00


In [ ]:
min_token_length = 90
for row in df[df["chunk_token_count"] <= min_token_length].sample(5).iterrows():
    print(f'Chunk token count: {row[1]["chunk_token_count"]} | Text: {row[1]["sentence_chunk"]}')

Chunk token count: 7.25 | Text: Arsenal’s goal scoring record
Chunk token count: 84.25 | Text: After Gabriel had a tap-in disallowed for being offside in the 59th minute, he headed in a corner from Gabriel Martinelli three minutes later. Arsenal continued to apply pressure to Southampton in the search of a fourth goal, but both Martinelli and Saka hit the posts with their shots. Meanwhile, Aaron Ramsdale made six saves to secure
Chunk token count: 21.0 | Text: Before halftime, Xhaka scored a spectacular free kick to increase Arsenal's lead. In
Chunk token count: 51.75 | Text: At Highbury like at Everton he did not get many chances, staying in the shadows owing to the excellence of the respective first choices, which was never better displayed than during the 1970/71 double season
Chunk token count: 15.25 | Text: 2006 The club move to the Emirates Stadium.   Club nicknames:


In [ ]:
pages_and_chunks_over_min_token_len = df[df["chunk_token_count"] > min_token_length].to_dict(orient="records")
pages_and_chunks_over_min_token_len[:2]

[{'page_number': 0,
  'sentence_chunk': 'Managed by Mikel Arteta in his first full season, Arsenal finished eighth in their Premier League season, and following eliminations from the FA Cup, EFL Cup and UEFA Europa League, they failed to qualify for any European competition for the first time since the 1994–95 season.   Arsenal’s 2019–20 campaign had seen defensive frailties and a lack of creativity lead to a disappointing eighth-place finish in the league, the club’s lowest for a quarter of a century, but a successful FA Cup run and increasing promise under new coach and former cup-winning captain Mikel Arteta gave them increased optimism for the season ahead. The season had been defined by a three-month lull between March and June, caused by the COVID-19 pandemic; indeed, it was head coach Arteta’s positive test that led to the season’s pause, and when it returned, it was behind closed doors. The upcoming season was expected to continue in this manner. The league fixtures were releas

In [ ]:
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer(model_name_or_path="all-mpnet-base-v2",
                                      device="cuda")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
%%time
embedding_model.to("cuda")
for item in tqdm(pages_and_chunks_over_min_token_len):
    item["embedding"] = embedding_model.encode(item["sentence_chunk"])

  0%|          | 0/228 [00:00<?, ?it/s]

CPU times: user 6.79 s, sys: 78.1 ms, total: 6.87 s
Wall time: 7.16 s


In [ ]:
text_chunks = [item["sentence_chunk"] for item in pages_and_chunks_over_min_token_len]

In [ ]:
%%time
text_chunk_embeddings = embedding_model.encode(text_chunks,
                                               batch_size=512,
                                               convert_to_tensor=True)
text_chunk_embeddings

CPU times: user 5.45 s, sys: 34 ms, total: 5.48 s
Wall time: 5.2 s


tensor([[-0.0445, -0.0009,  0.0163,  ...,  0.0247,  0.0196, -0.0073],
        [-0.0527, -0.0508,  0.0228,  ...,  0.0067, -0.0177, -0.0004],
        [-0.0505, -0.0733,  0.0251,  ..., -0.0350,  0.0423, -0.0028],
        ...,
        [-0.0142, -0.0039,  0.0350,  ..., -0.0044, -0.0207,  0.0187],
        [-0.0067,  0.0420,  0.0395,  ...,  0.0012, -0.0060,  0.0138],
        [ 0.0176, -0.0129, -0.0031,  ..., -0.0099, -0.0085, -0.0126]],
       device='cuda:0')

In [ ]:
text_chunks_and_embeddings_df = pd.DataFrame(pages_and_chunks_over_min_token_len)
embeddings_df_save_path = "text_chunks_and_embeddings_df.csv"
text_chunks_and_embeddings_df.to_csv(embeddings_df_save_path, index=False, escapechar='\\')

In [ ]:
text_chunks_and_embedding_df_load = pd.read_csv(embeddings_df_save_path)
text_chunks_and_embedding_df_load.tail()

,page_number,sentence_chunk,chunk_char_count,chunk_word_count,chunk_token_count,embedding
223,123,From the GreenOnScreen.co.uk player biography ...,913,167,228.25,[-1.00467849e-04 -8.57603550e-02 5.28866388e-...
224,124,Miller celebrating with goal scorer Alan Smith...,3257,591,814.25,[-3.39407809e-02 -4.58823629e-02 4.73073609e-...
225,124,Immortalised at Anfield 89 on the touchline wi...,1387,245,346.75,[-1.42496154e-02 -3.91553249e-03 3.49574871e-...
226,125,Bobby Robson Welcomes Paul Mariner to Ipswich!...,3843,709,960.75,[-6.71308720e-03 4.20162082e-02 3.94619890e-...
227,125,"usually gets asked. Well, here is the answer –...",628,121,157.00,[ 1.75904986e-02 -1.29140932e-02 -3.05678160e-...


In [ ]:
import random
import torch
import numpy as np
import pandas as pd

device = "cuda" if torch.cuda.is_available() else "cpu"
text_chunks_and_embedding_df = pd.read_csv("/content/text_chunks_and_embeddings_df.csv")
text_chunks_and_embedding_df["embedding"] = text_chunks_and_embedding_df["embedding"].apply(lambda x: np.fromstring(x.strip("[]"), sep=" "))
pages_and_chunks = text_chunks_and_embedding_df.to_dict(orient="records")
embeddings = torch.tensor(np.array(text_chunks_and_embedding_df["embedding"].tolist()), dtype=torch.float32).to(device)
embeddings.shape

torch.Size([228, 768])

In [ ]:
from sentence_transformers import util

In [ ]:
query = "Who is Thierry Henry"
print(f"Query: {query}")
query_embedding = embedding_model.encode(query, convert_to_tensor=True)
from time import perf_counter as timer

start_time = timer()
dot_scores = util.dot_score(a=query_embedding, b=embeddings)[0]
end_time = timer()

print(f"Time take to get scores on {len(embeddings)} embeddings: {end_time-start_time:.5f} seconds.")

top_results_dot_product = torch.topk(dot_scores, k=5)
top_results_dot_product

Query: Who is Thierry Henry
Time take to get scores on 228 embeddings: 0.00270 seconds.


torch.return_types.topk(
values=tensor([0.5817, 0.4578, 0.4528, 0.4402, 0.4382], device='cuda:0'),
indices=tensor([152,  73, 160, 133,  78], device='cuda:0'))

In [ ]:
larger_embeddings = torch.randn(100*embeddings.shape[0], 768).to(device)
print(f"Embeddings shape: {larger_embeddings.shape}")

start_time = timer()
dot_scores = util.dot_score(a=query_embedding, b=larger_embeddings)[0]
end_time = timer()

print(f"Time take to get scores on {len(larger_embeddings)} embeddings: {end_time-start_time:.5f} seconds.")

Embeddings shape: torch.Size([22800, 768])
Time take to get scores on 22800 embeddings: 0.02167 seconds.


In [ ]:
import textwrap

def print_wrapped(text, wrap_length=80):
    wrapped_text = textwrap.fill(text, wrap_length)
    print(wrapped_text)

In [ ]:
print(f"Query: '{query}'\n")
print("Results:")
for score, idx in zip(top_results_dot_product[0], top_results_dot_product[1]):
    print(f"Score: {score:.4f}")
    print("Text:")
    print_wrapped(pages_and_chunks[idx]["sentence_chunk"])
    print(f"Page number: {pages_and_chunks[idx]['page_number']}")
    print("\n")

Query: 'Who is Thierry Henry'

Results:
Score: 0.5817
Text:
Robert Pires is for sure one of the most successful foreign players the Premier
League has seen.   Arsene Wenger brought Pires to Arsenal in 2000. The Frenchman
quickly adapted to the style of play of Wenger and soon started to show his
class.   Along with Thiery Henry and Patrick Vieira, Pires dominated the pitch
every time they stepped on it.   Arsenal’s Invincibles was something different.
The team was full of one of the most talented footballers of all time.   Patrick
Vieira is often regarded as one of the best midfielders of his generation. A
true Arsenal legend, Vieira was known for his aggressive and highly competitive
style of play.   Patrick Vieira played for Arsenal for nine years, during which
he won the Premier League three times and the FA Cup four times.   Dennis
Bergkamp arrived at Arsenal in 1995, and stayed with the club for 11 years,
until his retirement in 2006. Bergkamp completely changed the DNA of the clu

In [ ]:
def retrieve_relevant_resources(query: str,
                                embeddings: torch.tensor,
                                model: SentenceTransformer=embedding_model,
                                n_resources_to_return: int=13,  # Update the default value to 15
                                print_time: bool=True):
    query_embedding = model.encode(query,
                                   convert_to_tensor=True)
    start_time = timer()
    dot_scores = util.dot_score(query_embedding, embeddings)[0]
    end_time = timer()

    if print_time:
        print(f"[INFO] Time taken to get scores on {len(embeddings)} embeddings: {end_time-start_time:.5f} seconds.")

    scores, indices = torch.topk(input=dot_scores,
                                 k=n_resources_to_return)

    return scores, indices

def print_top_results_and_scores(query: str,
                                 embeddings: torch.tensor,
                                 pages_and_chunks: list[dict]=pages_and_chunks,
                                 n_resources_to_return: int=5):
    scores, indices = retrieve_relevant_resources(query=query,
                                                  embeddings=embeddings,
                                                  n_resources_to_return=n_resources_to_return)
    print(f"Query: {query}\n")
    print("Results:")
    for score, index in zip(scores, indices):
        print(f"Score: {score:.4f}")
        print_wrapped(pages_and_chunks[index]["sentence_chunk"])
        print(f"Page number: {pages_and_chunks[index]['page_number']}")
        print("\n")

In [ ]:
query = "Cups of Arsenal"
scores, indices = retrieve_relevant_resources(query=query,
                                              embeddings=embeddings)
scores, indices

[INFO] Time taken to get scores on 228 embeddings: 0.00005 seconds.


(tensor([0.6874, 0.6741, 0.6636, 0.6604, 0.6579], device='cuda:0'),
 tensor([ 89,  94,  78, 118, 189], device='cuda:0'))

In [ ]:
print_top_results_and_scores(query=query,
                             embeddings=embeddings)

[INFO] Time taken to get scores on 228 embeddings: 0.00006 seconds.
Query: Cups of Arsenal

Results:
Score: 0.6874
of the FA Cup and League Cup in 1992–93.[268] The 2003–04 season was the only
38-match league season unbeaten in English football history. A special gold
version of the Premier League trophy was commissioned and presented to the club
the following season.[269]    Other    When the FA Cup was the only national
football association competition available to Arsenal, the other football
association competitions were County Cups, and they made up many of the matches
the club played during a season.[265] Arsenal's first first-team trophy was a
County Cup, the inaugural Kent Senior Cup.[17] Arsenal became ineligible for the
London Cups when the club turned professional in 1891, and rarely participated
in County Cups after this.[19][271] Due to the club's original location within
the borders of both the London and Kent Football Associations,[272] Arsenal
competed in and won trophie

In [ ]:
use_quantization_config = False
model_id = "google/gemma-2b-it"

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) y
Token is valid (permission: read).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' credent

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers.utils import is_flash_attn_2_available
from transformers import BitsAndBytesConfig
quantization_config = BitsAndBytesConfig(load_in_4bit=True,
                                         bnb_4bit_compute_dtype=torch.float16)
if (is_flash_attn_2_available()) and (torch.cuda.get_device_capability(0)[0] >= 8):
  attn_implementation = "flash_attention_2"
else:
  attn_implementation = "sdpa"
print(f"[INFO] Using attention implementation: {attn_implementation}")
model_id = model_id
print(f"[INFO] Using model_id: {model_id}")
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_id)
llm_model = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path=model_id,
                                                 torch_dtype=torch.float16,
                                                 quantization_config=quantization_config if use_quantization_config else None,
                                                 low_cpu_mem_usage=False,
                                                 attn_implementation=attn_implementation)

if not use_quantization_config:
    llm_model.to("cuda")

[INFO] Using attention implementation: sdpa
[INFO] Using model_id: google/gemma-2b-it


tokenizer_config.json:   0%|          | 0.00/2.16k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/888 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [ ]:
llm_model

GemmaForCausalLM(
  (model): GemmaModel(
    (embed_tokens): Embedding(256000, 2048, padding_idx=0)
    (layers): ModuleList(
      (0-17): 18 x GemmaDecoderLayer(
        (self_attn): GemmaSdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): GemmaRotaryEmbedding()
        )
        (mlp): GemmaMLP(
          (gate_proj): Linear(in_features=2048, out_features=16384, bias=False)
          (up_proj): Linear(in_features=2048, out_features=16384, bias=False)
          (down_proj): Linear(in_features=16384, out_features=2048, bias=False)
          (act_fn): GELUActivation()
        )
        (input_layernorm): GemmaRMSNorm()
        (post_attention_layernorm): GemmaRMSNorm()
      )
    )
    (norm): GemmaRM

In [ ]:
def get_model_num_params(model: torch.nn.Module):
    return sum([param.numel() for param in model.parameters()])

get_model_num_params(llm_model)

2506172416

In [ ]:
def get_model_mem_size(model: torch.nn.Module):
    mem_params = sum([param.nelement() * param.element_size() for param in model.parameters()])
    mem_buffers = sum([buf.nelement() * buf.element_size() for buf in model.buffers()])
    model_mem_bytes = mem_params + mem_buffers
    model_mem_mb = model_mem_bytes / (1024**2)
    model_mem_gb = model_mem_bytes / (1024**3)

    return {"model_mem_bytes": model_mem_bytes,
            "model_mem_mb": round(model_mem_mb, 2),
            "model_mem_gb": round(model_mem_gb, 2)}
get_model_mem_size(llm_model)

{'model_mem_bytes': 5079453696, 'model_mem_mb': 4844.14, 'model_mem_gb': 4.73}

In [ ]:
input_text = "Tell me the history of the Arsenal football club by the decade from their inception in 1886. Include the trophies they won each season who the managers were and who are the legends that have played for Arsenal football club. Also include the history at Highbury and at the Emirates stadium."
print(f"Input text:\n{input_text}")
dialogue_template = [
    {"role": "user",
     "content": input_text}
]
prompt = tokenizer.apply_chat_template(conversation=dialogue_template,
                                       tokenize=False,
                                       add_generation_prompt=True)
print(f"\nPrompt (formatted):\n{prompt}")

Input text:
Tell me the history of the Arsenal football club by the decade from their inception in 1886. Include the trophies they won each season who the managers were and who are the legends that have played for Arsenal football club. Also include the history at Highbury and at the Emirates stadium.

Prompt (formatted):
<bos><start_of_turn>user
Tell me the history of the Arsenal football club by the decade from their inception in 1886. Include the trophies they won each season who the managers were and who are the legends that have played for Arsenal football club. Also include the history at Highbury and at the Emirates stadium.<end_of_turn>
<start_of_turn>model



In [ ]:
%%time
input_ids = tokenizer(prompt, return_tensors="pt").to("cuda")
print(f"Model input (tokenized):\n{input_ids}\n")
outputs = llm_model.generate(**input_ids,
                             max_new_tokens=4096)
print(f"Model output (tokens):\n{outputs[0]}\n")

Model input (tokenized):
{'input_ids': tensor([[     2,      2,    106,   1645,    108,  27445,    682,    573,   4281,
            576,    573,  37346,   9715,   5893,    731,    573,  19199,    774,
           1024,  72173,    575, 235248, 235274, 235321, 235321, 235318, 235265,
          29034,    573,  87785,    984,   2792,   1853,   3891,   1064,    573,
          20641,   1049,    578,   1064,    708,    573,  35399,    674,    791,
           6991,    604,  37346,   9715,   5893, 235265,   6793,   3707,    573,
           4281,    696,   4211,  13063,    578,    696,    573,  56921,  31759,
         235265,    107,    108,    106,   2516,    108]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],
       device='cuda:0')}

Model output (tokens):
tensor([     2

In [ ]:
outputs_decoded = tokenizer.decode(outputs[0])
print(f"Model output (decoded):\n{outputs_decoded}\n")

Model output (decoded):
<bos><bos><start_of_turn>user
Tell me the history of the Arsenal football club by the decade from their inception in 1886. Include the trophies they won each season who the managers were and who are the legends that have played for Arsenal football club. Also include the history at Highbury and at the Emirates stadium.<end_of_turn>
<start_of_turn>model
**1886-1900s:**

* Founded in 1886, Arsenal Football Club is one of the oldest football clubs in the world.
* The club initially played in the Football Alliance, a local amateur league.
* In 1901, Arsenal won their first major trophy, the London County Cup.
* The club continued to win trophies in the 1900s, including the FA Cup in 1903 and 1904.

**1910s-1920s:**

* The club faced financial difficulties in the 1910s, but they eventually returned to success.
* In 1921, the club won their first major trophy in the FA Cup, the First Division title.
* The club continued to win trophies in the 1920s, including the FA C

In [ ]:
print(f"Input text: {input_text}\n")
print(f"Output text:\n{outputs_decoded.replace(prompt, '').replace('<bos>', '').replace('<eos>', '')}")

Input text: Tell me the history of the Arsenal football club by the decade from their inception in 1886. Include the trophies they won each season who the managers were and who are the legends that have played for Arsenal football club. Also include the history at Highbury and at the Emirates stadium.

Output text:
**1886-1900s:**

* Founded in 1886, Arsenal Football Club is one of the oldest football clubs in the world.
* The club initially played in the Football Alliance, a local amateur league.
* In 1901, Arsenal won their first major trophy, the London County Cup.
* The club continued to win trophies in the 1900s, including the FA Cup in 1903 and 1904.

**1910s-1920s:**

* The club faced financial difficulties in the 1910s, but they eventually returned to success.
* In 1921, the club won their first major trophy in the FA Cup, the First Division title.
* The club continued to win trophies in the 1920s, including the FA Cup in 1923 and 1924.

**1930s-1940s:**

* Arsenal faced more f

In [ ]:
query_list = ["Tell me all about Arsenal history"]

In [ ]:
query = random.choice(query_list)
print(f"Query: {query}")

# Get relevant resources
scores, indices = retrieve_relevant_resources(query=query,
                                              embeddings=embeddings)

# Create a list of context items with scores
context_items = [pages_and_chunks[i] for i in indices]
for i, item in enumerate(context_items):
    item["score"] = scores[i].cpu().item()

Query: Tell me all about Arsenal history
[INFO] Time taken to get scores on 228 embeddings: 0.00006 seconds.


In [ ]:
def prompt_formatter(query: str,
                     context_items: list[dict]) -> str:
    context_with_scores = []
    for item in context_items:
        context_with_scores.append(f"[Score: {item['score']:.4f}, Page: {item['page_number']}] {item['sentence_chunk']}")
    context = " \n ".join(context_with_scores)

    base_prompt = """Generate an extremely comprehensive and exhaustive answer on the history of the Arsenal football club.

    Your response should be at least 5000 words long, divided into multiple detailed paragraphs, each containing 10-15 sentences. Provide in-depth and extensive details on every aspect of Arsenal's history, including but not limited to:

    - Founding and early years, including the club's origins and initial struggles
    - Comprehensive list of major achievements and trophies won throughout the club's history, with details on significant matches and seasons
    - Biographical information and achievements of legendary players and managers, their impact on the club's success, and their legacy
    - Detailed accounts of significant events, matches, and rivalries that shaped the club's history, including memorable goals, comebacks, and controversies
    - Comprehensive history of the club's stadiums, including Highbury and Emirates, with details on their construction, renovations, and cultural significance
    - In-depth overview of the club's youth academy and player development system, highlighting notable players who came through the ranks
    - Analysis of the club's global fanbase, cultural impact, and influence on the sport beyond just their on-field achievements

    Make sure to thoroughly cover all the information available in the context passages below, and supplement it with your own extensive knowledge and insights. Use a formal, authoritative, and informative writing style, with well-researched facts and statistics. Ensure that your sentences are well-structured, varied in length, and flow coherently.

    \nNow take everything as context from the answer and use the following context items to answer the user query:
    {context}
    \nRelevant passages: <extract relevant passages from the context here, including the page numbers>
    User query: {query}
    Answer:"""
    base_prompt = base_prompt.format(context=context, query=query)
    dialogue_template = [
        {"role": "user",
         "content": base_prompt}
    ]
    prompt = tokenizer.apply_chat_template(conversation=dialogue_template,
                                           tokenize=False,
                                           add_generation_prompt=True)
    return prompt

In [ ]:
query = random.choice(query_list)
print(f"Query: {query}")

# Get relevant resources
scores, indices = retrieve_relevant_resources(query=query,
                                              embeddings=embeddings,
                                              n_resources_to_return=13)  # Specify the desired number of sources

# Create a list of context items with scores
context_items = [pages_and_chunks[i] for i in indices]
for i, item in enumerate(context_items):
    item["score"] = scores[i].cpu().item()

# Print the number of sources being used
print(f"Number of sources used: {len(context_items)}")

# Format prompt with context items
prompt = prompt_formatter(query=query,
                          context_items=context_items)
print(prompt)

Query: Tell me all about Arsenal history
[INFO] Time taken to get scores on 228 embeddings: 0.00006 seconds.
Number of sources used: 13
<bos><start_of_turn>user
Generate an extremely comprehensive and exhaustive answer on the history of the Arsenal football club.

    Your response should be at least 5000 words long, divided into multiple detailed paragraphs, each containing 10-15 sentences. Provide in-depth and extensive details on every aspect of Arsenal's history, including but not limited to:

    - Founding and early years, including the club's origins and initial struggles
    - Comprehensive list of major achievements and trophies won throughout the club's history, with details on significant matches and seasons
    - Biographical information and achievements of legendary players and managers, their impact on the club's success, and their legacy
    - Detailed accounts of significant events, matches, and rivalries that shaped the club's history, including memorable goals, comeba

In [ ]:
%%time
input_ids = tokenizer(prompt, return_tensors="pt").to("cuda")
outputs = llm_model.generate(**input_ids,
                             temperature=0.7,
                             do_sample=True,
                             max_new_tokens=9000)
output_text = tokenizer.decode(outputs[0])
print(f"Query: {query}")
print(f"RAG answer:\n{output_text.replace(prompt, '')}")

OutOfMemoryError: CUDA out of memory. Tried to allocate 5.76 GiB. GPU 0 has a total capacity of 14.75 GiB of which 403.06 MiB is free. Process 2951 has 14.35 GiB memory in use. Of the allocated memory 8.43 GiB is allocated by PyTorch, and 5.79 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
def ask(query,
        temperature=0.7,
        max_new_tokens=9000,
        format_answer_text=True,
        return_answer_only=True):
    scores, indices = retrieve_relevant_resources(query=query,
                                                  embeddings=embeddings)
    context_items = [pages_and_chunks[i] for i in indices]
    for i, item in enumerate(context_items):
        item["score"] = scores[i].cpu()
    prompt = prompt_formatter(query=query,
                              context_items=context_items)
    input_ids = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = llm_model.generate(**input_ids,
                                 temperature=temperature,
                                 do_sample=True,
                                 max_new_tokens=max_new_tokens)
    output_text = tokenizer.decode(outputs[0])

    if format_answer_text:
        answer_sentences = output_text.replace(prompt, "").replace("<bos>", "").replace("<eos>", "").split(". ")
        formatted_answer = []
        for sentence in answer_sentences:
            sentence = sentence.strip() + ". "
            if sentence:
                for item in context_items:
                    if sentence in item["sentence_chunk"]:
                        formatted_answer.append(f"[Source: Page {item['page_number']}] {sentence}")
                        break
                else:
                    formatted_answer.append(f"[Source: Generated] {sentence}")
        output_text = " ".join(formatted_answer)

    if return_answer_only:
        return output_text

    return output_text, context_items

In [ ]:
query = random.choice(query_list)
print(f"Query: {query}")
answer, context_items = ask(query=query,
                            temperature=0.7,
                            max_new_tokens=4096,
                            return_answer_only=False)
print(f"Answer:\n")
print_wrapped(answer)

Query: Tell me all about Arsenal history
[INFO] Time taken to get scores on 228 embeddings: 0.00006 seconds.
Answer:

[Source: Generated] ## Arsenal Football Club: A Historical Journey from Humble
Beginnings to Global Domination  The history of Arsenal Football Club is a
testament to the power of human ambition, resilience, and a never-say-die
spirit.  [Source: Generated] Founded by munitions workers in 1886, the club has
risen from humble beginnings to become one of the most successful and
influential clubs in the world.  **Early Years and Promotion to the First
Division:**  The foundation of Arsenal was laid by David Danskin, a Scottish
munitions worker, who gathered a group of friends to play football in Woolwich.
[Source: Generated] Despite being a club from an area known for rugby and
cricket, Arsenal quickly found itself drawn into the vibrant world of English
football.  [Source: Generated] Arsenal's first official match was played in
1893, marking the beginning of a remarkable c